# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,-0.3,0), (0,0,0)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((0,0.3,0), (0,0,0)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

connection_list
[False, False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.4,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.4,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add wp

In [7]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.3,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.3,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.3,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [8]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="indy0_tcp", dims=(0.1,0.07,0.02), 
                   center=(0.0,0.0,0.14), rpy=(0,np.pi,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="indy0_tcp", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=True, parent="brush_body")

## add box

In [9]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

# gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
#                    center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool, FixtureSlot

In [11]:
# gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
#                  dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
# pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=FixtureSlot, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="grip1", gname="grip1", rname="panda1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", rname="indy0", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject, FixturePoint, AbstractObject

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
# brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
#                              action_points_dict = {"handle": FixturePoint("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
#                                                    "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
box1 = pscene.create_object(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
# box2 = pscene.create_object(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [16]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [17]:
sweep1 = pscene.create_object(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_object(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_object(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [18]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion(mplan)
ppline.set_sampler(tplan)

## motion filters

In [19]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene, 
                      end_link_couple_dict= {
                          "indy0_tcp": ["indy0_tcp", "indy0_link6"],
                          "panda1_hand": ["panda1_hand", "panda1_link6"],
                          "base_link":["base_link"]})
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [20]:
mplan.motion_filters = checkers_all

In [21]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [22]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

('floor', 0, 0, 0)


In [23]:
pscene.subject_name_list

['box1', 'sweep1', 'sweep2', 'sweep3']

# Node Sampler

In [24]:
from pkg.planning.sampling.node_sampling import NodeSampler

In [25]:
tplan.new_node_sampler = NodeSampler(0.5)
tplan.parent_node_sampler = NodeSampler(0.5)

# CustomRule

In [26]:
class CustomRule:
    def __init__(self, pscene):
        self.pscene = pscene
        self.chain_dict = pscene.get_robot_chain_dict()
        
    def __call__(self, tplan, snode_src, snode_new, connection_result):
        if snode_src is not None:
            diff_sidxes = np.where([ ntem_s != ntem_g for ntem_s, ntem_g in zip(snode_src.state.node, snode_new.state.node)])[0]
            if len(diff_sidxes)==0:
                return False, None
            diff_sidx = diff_sidxes[0]
            diff_sname = pscene.subject_name_list[diff_sidx]
            diff_subject = pscene.subject_dict[diff_sname]
            if isinstance(diff_subject, AbstractObject):
                link_name1 = snode_src.state.state_param[diff_sname][0]
                link_name2 = snode_new.state.state_param[diff_sname][0]
                rname_candis = [rname for rname, chain_vals in self.chain_dict.items() if 
                               link_name1 in chain_vals['link_names'] or link_name2 in chain_vals['link_names']]
                if len(rname_candis)==0:
                    print("no robot candis")
                    return False, None
                else:
                    newstate = snode_new.state.copy(self.pscene)
                    jidxes = self.pscene.combined_robot.idx_dict[rname_candis[0]]
                    newstate.Q[jidxes] = self.pscene.combined_robot.home_pose[jidxes]
                    print("try: {}".format(newstate.node))
                    return True, newstate
            else:
                return False, None
        else:
            return False, None

In [27]:
tplan.custom_rule = CustomRule(pscene)

In [28]:
gtimer.reset()
goal_nodes = [("floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.sort_schedule(schedules)
snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
=============== try extend to goal ('floor', 0, 0, 0) -> ('floor', 0, 0, 1) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('grip1', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
=============== try extend to goal ('floor', 1, 0, 0) -> ('floor', 2, 0, 0) =================
node: ('floor', 0, 0, 0)->('floor', 1, 0, 0) = success
=============== try extend to goal ('floor', 1, 0, 0) -> ('floor', 2, 0, 0) =================
node: ('floor', 0, 0, 0)->('floor', 1, 0, 0) = success
branching: 0->2 (0.09/300.0 s, steps/err: 16(51.2340068817 ms)/0.00160820151184)
branching: 0->1 (0.09/300.0 s, steps/err: 12(37.9128456116 ms)/0.00196771576939)
=============== try extend to goal ('floor', 1, 0, 0) -> ('floor', 2, 0, 0) =================
node: ('floor', 0, 0, 0

node: ('floor', 0, 0, 2)->('floor', 1, 0, 2) = success
branching: 17->21 (1.38/300.0 s, steps/err: 22(71.4781284332 ms)/0.00169444580235)
node: ('floor', 0, 0, 1)->('floor', 0, 0, 2) = fail
try: ('grip1', 0, 0, 2)
node: ('floor', 0, 0, 1)->('floor', 0, 0, 2) = fail
=============== try extend to goal ('floor', 1, 0, 2) -> ('floor', 2, 0, 2) =================
node: ('floor', 0, 0, 2)->('grip1', 0, 0, 2) = success
branching: 17->22 (1.43/300.0 s, steps/err: 22(157.811164856 ms)/0.0014310924021)
node: ('floor', 0, 0, 2)->('floor', 1, 0, 2) = success
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
branching: 17->23 (1.45/300.0 s, steps/err: 20(143.808841705 ms)/0.00138514663569)
node: ('floor', 0, 0, 0)->('grip1', 0, 0, 0) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = fail
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
node: ('grip1', 0, 0, 2)->('grip1', 0, 0, 2) = success
=============== try extend to goal ('floor', 1, 0, 2) -> ('floor', 2, 0, 2) =================
branchin

node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 0) = fail
branching: 34->41 (2.74/300.0 s, steps/err: 14(121.666193008 ms)/0.00179376990557)
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 34->42 (2.84/300.0 s, steps/err: 14(86.1699581146 ms)/0.00149527856181)
node: ('floor', 0, 1, 2)->('floor', 0, 2, 2) = fail
node: ('floor', 0, 1, 2)->('floor', 0, 2, 2) = fail
=============== try extend to goal ('floor', 0, 1, 0) -> ('floor', 0, 2, 0) =================
node: ('floor', 1, 0, 2)->('floor', 2, 0, 2) = fail
node: ('grip1', 1, 0, 2)->('grip1', 2, 0, 2) = fail
node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = success
node: ('floor', 1, 0, 2)->('floor', 2, 0, 2) = fail
branching: 0->43 (2.92/300.0 s, steps/err: 14(142.838001251 ms)/0.00119758250549)
node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 1) = success
=============== try extend to goal ('grip1', 0, 2, 2) -> ('grip1', 1, 2, 2) =================
=============== try extend to goal ('grip1', 0, 1, 2) -> ('grip1', 0, 2, 2) ===========

branching: 45->62 (3.93/300.0 s, steps/err: 59(966.310024261 ms)/0.0010802476173)
branching: 55->63 (3.93/300.0 s, steps/err: 11(206.307888031 ms)/0.0011796854689)
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = fail
node: ('floor', 0, 1, 0)->('floor', 0, 2, 0) = fail
=============== try extend to goal ('grip1', 2, 1, 2) -> ('grip1', 2, 2, 2) =================
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = fail
node: ('grip1', 2, 0, 2)->('grip1', 2, 1, 2) = success
branching: 48->64 (4.13/300.0 s, steps/err: 6(56.4959049225 ms)/0.00160899742859)
node: ('grip1', 1, 0, 2)->('grip1', 2, 0, 2) = fail
=============== try extend to goal ('floor', 0, 1, 2) -> ('floor', 0, 2, 2) =================
=============== try extend to goal ('grip1', 1, 2, 2) -> ('grip1', 2, 2, 2) =================
node: ('grip1', 0, 2, 2)->('grip1', 1, 2, 2) = success
=============== try extend to goal ('floor', 0, 1, 2) -> ('floor', 0, 2, 2) =================
node: ('floor', 0, 0, 2)->('floor', 0, 1, 2) = success
node:

branching: 77->82 (5.22/300.0 s, steps/err: 16(105.149030685 ms)/0.00113018683827)
branching: 77->83 (5.22/300.0 s, steps/err: 16(79.3449878693 ms)/0.00112051536466)
=============== try reaching goal from ('floor', 1, 2, 2) =================
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = success
branching: 81->84 (5.34/300.0 s, steps/err: 16(66.0719871521 ms)/0.00182052237614)
node: ('grip1', 2, 2, 2)->('floor', 2, 2, 2) = fail
node: ('grip1', 1, 2, 2)->('grip1', 2, 2, 2) = fail
node: ('grip1', 1, 0, 0)->('grip1', 2, 0, 0) = fail
node: ('grip1', 2, 1, 2)->('grip1', 2, 2, 2) = fail
node: ('grip1', 1, 2, 2)->('grip1', 2, 2, 2) = fail
node: ('floor', 0, 1, 2)->('floor', 0, 2, 2) = fail
=============== try reaching goal from ('floor', 1, 2, 2) =================
=============== try extend to goal ('grip1', 0, 2, 1) -> ('grip1', 0, 2, 2) =================
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = success
branching: 81->85 (5.51/300.0 s, steps/err: 17(78.5140991211 ms)/0.00121662697697)


...

branching: 306->369 (26.17/300.0 s, steps/err: 17(272.668123245 ms)/0.00143742507215)
node: ('floor', 1, 2, 2)->('floor', 2, 2, 2) = fail
=============== try reaching goal from ('floor', 1, 2, 2) =================
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = success
node: ('grip1', 2, 1, 2)->('grip1', 2, 2, 2) = fail
branching: 331->370 (26.27/300.0 s, steps/err: 18(252.025127411 ms)/0.000753357586739)
=============== try extend to goal ('floor', 1, 0, 2) -> ('floor', 2, 0, 2) =================
node: ('floor', 0, 0, 2)->('floor', 1, 0, 2) = success
=============== try extend to goal ('floor', 0, 0, 0) -> ('floor', 0, 0, 1) =================
node: ('floor', 1, 2, 2)->('floor', 2, 2, 2) = fail
node: ('grip1', 0, 1, 2)->('grip1', 0, 2, 2) = fail
branching: 17->371 (26.33/300.0 s, steps/err: 20(98.8931655884 ms)/0.00184512153439)
node: ('floor', 0, 0, 0)->('floor', 0, 0, 0) = success
=============== try reaching goal from ('floor', 1, 2, 2) =================
=============== try extend to

node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = success
branching: 331->388 (27.7/300.0 s, steps/err: 14(160.908937454 ms)/0.00146224831484)
node: ('grip1', 2, 0, 1)->('grip1', 2, 0, 2) = fail
node: ('grip1', 2, 1, 2)->('grip1', 2, 2, 2) = fail
node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = success
branching: 231->389 (27.82/300.0 s, steps/err: 11(102.874040604 ms)/0.00126348510498)
node: ('grip1', 0, 0, 0)->('grip1', 0, 0, 1) = success
node: ('grip1', 0, 0, 0)->('grip1', 1, 0, 0) = success
branching: 34->390 (27.82/300.0 s, steps/err: 11(94.428062439 ms)/0.00155494728446)
=============== try extend to goal ('grip1', 1, 2, 2) -> ('grip1', 2, 2, 2) =================
node: ('floor', 2, 2, 0)->('floor', 2, 2, 0) = fail
branching: 74->392 (27.83/300.0 s, steps/err: 12(59.1509342194 ms)/0.00175087754414)
node: ('grip1', 0, 2, 2)->('grip1', 1, 2, 2) = success
branching: 46->391 (27.85/300.0 s, steps/err: 9(56.4351081848 ms)/0.0017450989913)


node: ('grip1', 2, 0, 0)->('grip1', 2, 0, 1) = success
node: ('grip1', 0, 2, 0)->('grip1', 1, 2, 0) = success
=============== try extend to goal ('grip1', 1, 2, 2) -> ('grip1', 2, 2, 2) =================
branching: 213->410 (28.93/300.0 s, steps/err: 11(76.5030384064 ms)/0.00122946079364)
branching: 77->409 (28.91/300.0 s, steps/err: 23(168.24388504 ms)/0.00191742243302)
node: ('grip1', 0, 2, 2)->('grip1', 1, 2, 2) = success
node: ('grip1', 1, 2, 2)->('grip1', 2, 2, 2) = fail
branching: 50->411 (28.95/300.0 s, steps/err: 16(123.375892639 ms)/0.00236235164148)
node: ('grip1', 0, 1, 2)->('grip1', 0, 2, 2) = fail
node: ('floor', 1, 2, 2)->('floor', 2, 2, 2) = fail
node: ('grip1', 2, 2, 0)->('floor', 2, 2, 0) = fail
node: ('grip1', 0, 0, 0)->('grip1', 0, 1, 0) = fail
=============== try reaching goal from ('floor', 2, 2, 1) =================
node: ('grip1', 0, 0, 0)->('grip1', 1, 0, 0) = success
node: ('floor', 0, 0, 0)->('floor', 0, 1, 0) = fail
node: ('floor', 2, 2, 0)->('floor', 2, 2, 1

node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
=============== try reaching goal from ('floor', 1, 2, 2) =================
=============== try extend to goal ('floor', 0, 0, 1) -> ('floor', 0, 0, 2) =================
node: ('floor', 0, 2, 2)->('floor', 1, 2, 2) = success
node: ('floor', 0, 0, 0)->('floor', 0, 0, 1) = success
branching: 331->429 (30.44/300.0 s, steps/err: 19(121.064901352 ms)/0.00184840530721)
=============== try extend to goal ('grip1', 2, 2, 1) -> ('grip1', 2, 2, 2) =================
branching: 198->428 (30.44/300.0 s, steps/err: 17(231.653928757 ms)/0.00220821710569)
node: ('grip1', 0, 2, 1)->('grip1', 0, 2, 2) = fail
node: ('grip1', 2, 2, 0)->('grip1', 2, 2, 1) = success
branching: 291->430 (30.51/300.0 s, steps/err: 10(42.897939682 ms)/0.00154569614026)
=============== try extend to goal ('floor', 0, 0, 1) -> ('floor', 0, 0, 2) =================
node: ('floor', 0, 0, 0)->('floor', 0, 0, 1) = success
node: ('grip1', 0, 0, 1)->('grip1', 0, 0, 2) = fail
branching

======================= terminated 17: first answer acquired from other agent ===============================
node: ('grip1', 0, 1, 2)->('grip1', 0, 2, 2) = fail
======================= terminated 9: first answer acquired from other agent ===============================
node: ('grip1', 0, 1, 0)->('grip1', 0, 2, 0) = fail
======================= terminated 13: first answer acquired from other agent ===============================
node: ('grip1', 0, 1, 2)->('grip1', 0, 2, 2) = success
branching: 441->446 (32.09/300.0 s, steps/err: 86(579.126834869 ms)/0.00154312783106)
======================= terminated 2: first answer acquired from other agent ===============================
node: ('grip1', 0, 1, 0)->('grip1', 0, 2, 0) = fail
======================= terminated 8: first answer acquired from other agent ===============================
node: ('floor', 2, 2, 1)->('floor', 2, 2, 2) = fail
======================= terminated 10: first answer acquired from other agent ==========================

In [29]:
print(gtimer)

plan: 	31766.0 ms/1 = 31765.633 ms (31765.633/31765.633)



## play searched plan

In [30]:
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [31]:
snode_schedule = ppline.add_return_motion(snode_schedule)

In [32]:
ppline.play_schedule(snode_schedule, period=0.03)

In [33]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

schedule length: 12
0: ('floor', 0, 0, 0)
1: ('grip1', 0, 0, 0)
2: ('grip1', 0, 0, 0)
3: ('grip1', 1, 0, 0)
4: ('grip1', 2, 0, 0)
5: ('grip1', 2, 0, 1)
6: ('grip1', 2, 0, 2)
7: ('grip1', 2, 1, 2)
8: ('grip1', 2, 2, 2)
9: ('floor', 2, 2, 2)
10: ('floor', 2, 2, 2)
11: ('floor', 2, 2, 2)


## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}

## extend preserving goal-matching items

In [25]:
print(gtimer)

plan: 	207291.0 ms/10 = 20729.062 ms (9041.162/45383.357)



## extend only no reservation

In [25]:
print(gtimer)

plan: 	270954.0 ms/10 = 27095.38 ms (6255.591/68399.08)



## no extension

In [25]:
print(gtimer)

plan: 	226519.0 ms/10 = 22651.91 ms (8120.631/69353.952)



## extend_toward goal

In [30]:
print(gtimer)

plan: 	204994.0 ms/10 = 20499.448 ms (8105.979/60408.919)

